In [1]:
import cPickle as pickle
import pandas as pd
import math
import time
from pandas import DataFrame

from selenium import webdriver
base_path = '/home/hareesh/Research/big_data/project/airbnb/'
driver = webdriver.Firefox(executable_path=base_path+'notebooks/geckodriver')
driver.set_page_load_timeout(30)
driver.maximize_window()

In [2]:
df_listings = pd.read_csv(base_path+'inside_airbnb/toronto/listings.csv')
df_listings.shape

(12029, 95)

In [3]:
def open_url_click(driver):
    
    # Note:XPaths are subject to change on Airbnb site 
    
    # Click more button under Amenities Section
    try:
        driver.find_element_by_xpath('''//*[@id="details"]/div/div/div/div/div[4]/div[2]/div[1]/div/div[1]/button''').click()
    except:
        driver.find_element_by_xpath('''//*[@id="details"]/div/div/div/div/div[5]/div[2]/div[1]/div/div[1]/button''').click()
    # Some pages have it in a different xpath as mentioned in except block

    # Click more button under Description
    try:
        driver.find_element_by_xpath('''//*[@id="house-rules"]/div[2]/div[2]/div/span/button''').click()
    except:
        pass

    # Click more button under House Rules
    try:
        driver.find_element_by_xpath('''//*[@id="description"]/div[2]/div/div/span/button''').click()
    except:
        pass



In [ ]:
%%time
# url = df_listings.ix[85, 'listing_url']

page_saved_dict = {}
amenity_dict_available = {}
amenity_dict_not_available = {}
# error_log = ""

for k in range(df_listings.shape[0]):

    listing_id = df_listings.ix[k, 'id']
    url = df_listings.ix[k, 'listing_url']
#     url = "https://www.airbnb.com/rooms/9842943"
    
    try:
        driver.get(url)
        open_url_click(driver)

        # Number of travellers who saved this listing to wish list 
        num_saved = driver.find_element_by_xpath('''//*[@id="summary"]/div/div/div[2]/div[3]/div/div[3]/div[1]/div/div/div/small/span''')
        page_saved_dict[listing_id] = int(num_saved.text.split()[0])

        amenities = driver.find_elements_by_class_name("col-sm-6")
        amenity_ids = [4, 5] # These are the items in the HTML element list that we are interested in
        amenity_list = [item.text for i, item in enumerate(amenities) if i in amenity_ids]
        nested_list = [item.split('\n') for item in amenity_list]
        amenity_list = [item for sublist in nested_list for item in sublist]

        # The ones with space before the amenity is spelled out refers to the ones available at the house
        amenity_dict_available[listing_id] = [item.strip() for item in amenity_list if item[0] == ' ']
        amenity_dict_not_available[listing_id] = [item.strip() for item in amenity_list if item[0] != ' ']

    except Exception as e: # Sometimes the listing would have been removed and the page gets redirected
        print str(k) + " " + url + " " + str(e) + "\n"
        continue 
        
    if k%100 == 0:
        print k, "Complete"

In [8]:
with open('temp_pickle_files/toronto_listing_amenities_1', 'wb') as f:
    pickle.dump(amenity_dict_available, f)
    pickle.dump(amenity_dict_not_available, f)
    pickle.dump(page_saved_dict, f)

In [11]:
with open('temp_pickle_files/toronto_listing_amenities_1', 'rb') as f:
    amenity_dict_available = pickle.load(f)
    amenity_dict_not_available = pickle.load(f)
    page_saved_dict = pickle.load(f)

In [16]:
# Merge multiple dictionaries from different pickle files (the scraping operation was done in multiple stages) into one
amenity_dict_available = {}
amenity_dict_not_available = {}
page_saved_dict = {}

for i in range(1,9):
    with open('temp_pickle_files/toronto_listing_amenities_'+str(i), 'rb') as f:
        x = pickle.load(f)
        y = pickle.load(f)
        z = pickle.load(f)

        amenity_dict_available = dict(amenity_dict_available.items() + x.items())
        amenity_dict_not_available = dict(amenity_dict_not_available.items() + y.items())
        page_saved_dict = dict(page_saved_dict.items() + z.items())

In [17]:
with open('toronto_listing_amenities', 'wb') as f:
    pickle.dump(amenity_dict_available, f)
    pickle.dump(amenity_dict_not_available, f)
    pickle.dump(page_saved_dict, f)